## OpenCV 사용해서 U-Net 학습에 필요한 True mask 이미지 생성

In [11]:
import cv2
import numpy as np
import os
import tensorflow as tf

def parse_tfrecord(example):
    features = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'img_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64),
    }

    example = tf.io.parse_single_example(example, features)
    
    # 이미지 경로 수정
    image_path = tf.strings.join([example['img_path'], tf.constant('/'), example['Raw_data_ID']])

    print(image_path)

    # 좌표를 리스트로 변환
    x_coords = tf.sparse.to_dense(example['x_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    y_coords = tf.sparse.to_dense(example['y_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    
    # 좌표 튜플로 반환
    coords_pairs = (x_coords, y_coords)
    return image_path, example['lesions'], coords_pairs

# TFRecord 파일 불러오는 함수
def load_dataset(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord)
    return dataset





def create_true_mask(image_path, x_coords, y_coords, save_dir):
    # 이미지 경로를 문자열로 변환
    image_path = image_path.numpy().decode('utf-8')
    # 이미지 불러오기 (높이, 너비 정보만 사용)
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    
    # 다각형 좌표를 NumPy 배열로 변환
    pts = np.array(list(zip(x_coords.numpy(), y_coords.numpy())), np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    # 마스크 이미지 생성 (배경: 회색(128), 좌표 영역: 초록색(0, 255, 0))
    mask = np.full((height, width, 3), 128, dtype=np.uint8)
    cv2.fillPoly(mask, [pts], color=(0, 255, 0))
    
    # 마스크 이미지 저장
    filename = os.path.basename(image_path)
    mask_save_path = os.path.join(save_dir, f"mask_{filename}")
    cv2.imwrite(mask_save_path, mask)
    print(f"True mask 이미지가 생성되었습니다. 저장된 경로: {mask_save_path}")




tfrecord_file = './train_symptoms_tfrecords/train_symptoms_A6.tfrecord'

dataset = load_dataset(tfrecord_file)

for example in dataset:
    image_path, lesions, coords_pairs = example
    print('img_path:', image_path.numpy().decode('utf-8'))
    print("Lesion:", lesions.numpy().decode('utf-8'))

    x_coords, y_coords = coords_pairs
    print("X Coords:", x_coords.numpy())
    print("Y Coords:", y_coords.numpy())

    # 저장할 디렉토리 생성
    save_dir = './train_true_mask_images_A6'
    os.makedirs(save_dir, exist_ok=True)
    create_true_mask(image_path, x_coords, y_coords, save_dir)

Tensor("StringJoin:0", shape=(), dtype=string)
img_path: ../Segmentation/train_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_28_039246.jpg
Lesion: A6
X Coords: [661 564 504 505 545 603 662 733 802 853 882 903 907 872 831 788 661]
Y Coords: [150 183 261 337 406 459 475 470 460 440 417 378 319 222 183 156 150]


AttributeError: 'NoneType' object has no attribute 'shape'

## train_A4부터 오류 발생해 확인해 보니, JSON 파일에 RAW_DATA_ID가 이미지와 맞지 않는 경우 있음

In [20]:
import cv2
import numpy as np
import os
import tensorflow as tf

# 실패한 이미지 경로를 저장할 리스트
failed_image_paths = []

def parse_tfrecord(example):
    features = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'img_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64),
    }

    example = tf.io.parse_single_example(example, features)
    
    # 이미지 경로 수정
    image_path = tf.strings.join([example['img_path'], tf.constant('/'), example['Raw_data_ID']])

    print(image_path)

    # 좌표를 리스트로 변환
    x_coords = tf.sparse.to_dense(example['x_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    y_coords = tf.sparse.to_dense(example['y_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    
    # 좌표 튜플로 반환
    coords_pairs = (x_coords, y_coords)
    return image_path, example['lesions'], coords_pairs

# TFRecord 파일 불러오는 함수
def load_dataset(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord)
    return dataset

def create_true_mask(image_path, x_coords, y_coords, save_dir):
    global failed_image_paths
    # 이미지 경로를 문자열로 변환
    image_path = image_path.numpy().decode('utf-8')
    # 이미지 불러오기 (높이, 너비 정보만 사용)
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        failed_image_paths.append(image_path)
        return
    
    height, width, _ = image.shape
    
    # 다각형 좌표를 NumPy 배열로 변환
    pts = np.array(list(zip(x_coords.numpy(), y_coords.numpy())), np.int32)
    pts = pts.reshape((-1, 1, 2))
    
    # 마스크 이미지 생성 (배경: 회색(128), 좌표 영역: 초록색(0, 255, 0))
    mask = np.full((height, width, 3), 128, dtype=np.uint8)
    cv2.fillPoly(mask, [pts], color=(0, 255, 0))
    
    # 마스크 이미지 저장
    filename = os.path.basename(image_path)
    mask_save_path = os.path.join(save_dir, f"mask_{filename}")
    cv2.imwrite(mask_save_path, mask)
    print(f"True mask 이미지가 생성되었습니다. 저장된 경로: {mask_save_path}")

tfrecord_file = './val_symptoms_tfrecords/val_symptoms_A6.tfrecord'

dataset = load_dataset(tfrecord_file)

for example in dataset:
    image_path, lesions, coords_pairs = example
    print('img_path:', image_path.numpy().decode('utf-8'))
    print("Lesion:", lesions.numpy().decode('utf-8'))

    x_coords, y_coords = coords_pairs
    print("X Coords:", x_coords.numpy())
    print("Y Coords:", y_coords.numpy())

    # 저장할 디렉토리 생성
    save_dir = './val_true_mask_images_A6'
    os.makedirs(save_dir, exist_ok=True)
    create_true_mask(image_path, x_coords, y_coords, save_dir)


# 실패한 이미지 경로를 텍스트 파일로 저장
failed_image_paths_file = 'failed_val_image_paths_A6.txt'
with open(failed_image_paths_file, 'w') as f:
    for path in failed_image_paths:
        f.write(f"{path}\n")

print(f"이미지 로드가 실패한 경로들이 {failed_image_paths_file} 파일에 저장되었습니다.")


Tensor("StringJoin:0", shape=(), dtype=string)
img_path: ../Segmentation/val_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_496645.jpg
Lesion: A6
X Coords: [1609 1655 1691 1748 1754 1727 1646 1576 1525 1513 1513 1520 1609]
Y Coords: [1061 1064 1063 1039 1025 1007 1003 1000 1015 1025 1043 1046 1061]
True mask 이미지가 생성되었습니다. 저장된 경로: ./val_true_mask_images_A6\mask_IMG_D_A6_496645.jpg
img_path: ../Segmentation/val_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_496646.jpg
Lesion: A6
X Coords: [ 950  988 1037 1064 1033  941  835  830  845  854  854  856  950]
Y Coords: [625 608 581 524 458 430 440 496 574 595 595 596 625]
True mask 이미지가 생성되었습니다. 저장된 경로: ./val_true_mask_images_A6\mask_IMG_D_A6_496646.jpg
img_path: ../Segmentation/val_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_496647.jpg
Lesion: A6
X Coords: [467 541 577 617 625 605 548 521 502 467]
Y Coords: [550 556 551 521 487 442 437 461 496 550]
True mask 이미지가 생성되었습니다. 저장된 경로: ./val_true_mask_images_A6\mask_IMG_D_A6_496647.jpg
img_path: .

## U-Net 모델 학습에 필요한 mask 이미지 재생성 

In [8]:
import cv2
import numpy as np
import os
import tensorflow as tf

# 실패한 이미지 경로를 저장할 리스트
failed_image_paths = []

def parse_tfrecord(example):
    features = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'img_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64),
    }
    example = tf.io.parse_single_example(example, features)

    # 이미지 경로 수정
    image_path = tf.strings.join([example['img_path'], tf.constant('/'), example['Raw_data_ID']])

    # 좌표를 리스트로 변환
    x_coords = tf.sparse.to_dense(example['x_coords'], default_value=0)
    y_coords = tf.sparse.to_dense(example['y_coords'], default_value=0)

    # 유효한 좌표만 남기기
    valid_coords = tf.math.not_equal(x_coords, 0) & tf.math.not_equal(y_coords, 0)
    x_coords = tf.boolean_mask(x_coords, valid_coords)
    y_coords = tf.boolean_mask(y_coords, valid_coords)

    # 좌표 튜플로 반환
    coords_pairs = (x_coords, y_coords)
    return image_path, example['lesions'], coords_pairs

# TFRecord 파일 불러오는 함수
def load_dataset(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord)
    return dataset

def create_segmentation_mask(image_path, x_coords, y_coords, save_dir):
    global failed_image_paths

    # 이미지 경로를 문자열로 변환
    image_path = image_path.numpy().decode('utf-8')

    # 이미지 불러오기 (높이, 너비 정보만 사용)
    image = cv2.imread(image_path)
    if image is None:
        print(f"이미지를 불러올 수 없습니다: {image_path}")
        failed_image_paths.append(image_path)
        return

    height, width, _ = image.shape

    # 다각형 좌표를 NumPy 배열로 변환
    pts = np.array(list(zip(x_coords.numpy(), y_coords.numpy())), np.int32)
    pts = pts.reshape((-1, 1, 2))

    # 유효한 좌표가 있는지 확인
    if len(pts) == 0:
        print(f"유효한 좌표가 없습니다: {image_path}")
        return

    # 세그멘테이션 마스크 생성 (배경: 회색(128), 클래스 1: 초록색(0, 255, 0), 클래스 2: 파란색(255, 0, 0), 클래스 3: 빨간색(0, 0, 255))
    mask = np.full((height, width, 3), 128, dtype=np.uint8)

    # 클래스 1: 좌표값들을 연결했을 때 그 내부 영역
    cv2.fillPoly(mask, [pts], color=(0, 255, 0))

    # 클래스 2: 좌표를 선으로 연결했을 때 그 선 영역
    poly_mask = np.zeros_like(mask, dtype=np.uint8)
    cv2.polylines(poly_mask, [pts], True, (255, 0, 0), 2)
    mask[poly_mask[:, :, 0] == 255] = [255, 0, 0]

    # 클래스 3: 클래스 1과 2를 제외한 나머지 영역
    mask[np.logical_and(mask[:, :, 0] == 128, mask[:, :, 1] == 128, mask[:, :, 2] == 128)] = [0, 0, 255]

    # 마스크 이미지 저장
    filename = os.path.basename(image_path)
    mask_save_path = os.path.join(save_dir, f"mask_{filename}")
    cv2.imwrite(mask_save_path, mask)
    print(f"세그멘테이션 마스크 이미지가 생성되었습니다. 저장된 경로: {mask_save_path}")

tfrecord_file = './val_symptoms_tfrecords/val_symptoms_A6.tfrecord'
dataset = load_dataset(tfrecord_file)

for example in dataset.take(3):
    image_path, lesions, coords_pairs = example
    print('img_path:', image_path.numpy().decode('utf-8'))
    print("Lesion:", lesions.numpy().decode('utf-8'))
    x_coords, y_coords = coords_pairs
    print("X Coords:", x_coords.numpy())
    print("Y Coords:", y_coords.numpy())

    # 저장할 디렉토리 생성
    save_dir = './val_segmentation_mask_images_A6'
    os.makedirs(save_dir, exist_ok=True)

    create_segmentation_mask(image_path, x_coords, y_coords, save_dir)

# 실패한 이미지 경로를 텍스트 파일로 저장
failed_image_paths_file = 'failed_val_image_paths_A6.txt'
with open(failed_image_paths_file, 'w') as f:
    for path in failed_image_paths:
        f.write(f"{path}\n")
print(f"이미지 로드가 실패한 경로들이 {failed_image_paths_file} 파일에 저장되었습니다.")

img_path: ../Segmentation/val_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_496645.jpg
Lesion: A6
X Coords: [1609 1655 1691 1748 1754 1727 1646 1576 1525 1513 1513 1520 1609]
Y Coords: [1061 1064 1063 1039 1025 1007 1003 1000 1015 1025 1043 1046 1061]
세그멘테이션 마스크 이미지가 생성되었습니다. 저장된 경로: ./val_segmentation_mask_images_A6\mask_IMG_D_A6_496645.jpg
img_path: ../Segmentation/val_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_496646.jpg
Lesion: A6
X Coords: [ 950  988 1037 1064 1033  941  835  830  845  854  854  856  950]
Y Coords: [625 608 581 524 458 430 440 496 574 595 595 596 625]
세그멘테이션 마스크 이미지가 생성되었습니다. 저장된 경로: ./val_segmentation_mask_images_A6\mask_IMG_D_A6_496646.jpg
img_path: ../Segmentation/val_labelingdata/dog/skin/camera/symp/A6/IMG_D_A6_496647.jpg
Lesion: A6
X Coords: [467 541 577 617 625 605 548 521 502 467]
Y Coords: [550 556 551 521 487 442 437 461 496 550]
세그멘테이션 마스크 이미지가 생성되었습니다. 저장된 경로: ./val_segmentation_mask_images_A6\mask_IMG_D_A6_496647.jpg
이미지 로드가 실패한 경로들이 failed_val_ima